In [ ]:
#fakefriends.csv

user_id: 사용자 ID name: 사용자 이름 age: 사용자 나이 num_friends: 친구 수

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_04_RDDEXAM")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 16:29:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 16:29:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/05 16:29:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
import os
directory = os.path.join(os.getcwd(), "data")  # 현재 디렉토리의 data 폴더
filename  = "fakefriends.csv"
file_path = os.path.join(directory, filename)
# Spark RDD 생성
lines = spark.textFile("file:///" + file_path.replace("\\", "/"))

In [6]:
lines.take(3)

['0,Will,33,385', '1,Jean-Luc,26,2', '2,Hugh,55,221']

In [7]:
#나이와, 친구수만 추출
def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    
    return age, numFriends

In [8]:
rdd = lines.map(parseLine)
rdd.take(3)

[(33, 385), (26, 2), (55, 221)]

In [9]:
age_cnt_rdd = rdd.mapValues(lambda x: (x,1))
age_cnt_rdd.take(3) #age, num_friends, cnt

[(33, (385, 1)), (26, (2, 1)), (55, (221, 1))]

In [10]:
age_total_cnt = age_cnt_rdd.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
age_total_cnt.take(3)

[(33, (3904, 12)), (26, (4115, 17)), (55, (3842, 13))]

In [11]:
# 평균 친구수 구하기
age_avg_cnt = age_total_cnt.mapValues(lambda x: x[0]/x[1])
age_avg_cnt.take(5)

[(33, 325.3333333333333),
 (26, 242.05882352941177),
 (55, 295.53846153846155),
 (40, 250.8235294117647),
 (68, 269.6)]

In [12]:
# 가장 친구가 많은 사용자 찾기
rdd = lines.map(lambda row : row.split(","))
rdd.take(3)

[['0', 'Will', '33', '385'],
 ['1', 'Jean-Luc', '26', '2'],
 ['2', 'Hugh', '55', '221']]

In [15]:
struct_rdd = rdd.map(lambda row: (int(row[0]), row[1], int(row[2]), int(row[3])))
struct_rdd.take(3)

[(0, 'Will', 33, 385), (1, 'Jean-Luc', 26, 2), (2, 'Hugh', 55, 221)]

In [16]:
struct_rdd.max(key=lambda row:row[3])

(106, 'Beverly', 18, 499)

In [18]:
# 친구 수가 100 이상인 사용자 필터링
struct_rdd.filter(lambda row: row[3]>100).collect()

[(0, 'Will', 33, 385),
 (2, 'Hugh', 55, 221),
 (3, 'Deanna', 40, 465),
 (5, 'Weyoun', 59, 318),
 (6, 'Gowron', 37, 220),
 (7, 'Will', 54, 307),
 (8, 'Jadzia', 38, 380),
 (9, 'Hugh', 27, 181),
 (10, 'Odo', 53, 191),
 (11, 'Ben', 57, 372),
 (12, 'Keiko', 54, 253),
 (13, 'Jean-Luc', 56, 444),
 (16, 'Weyoun', 22, 323),
 (18, 'Jean-Luc', 45, 455),
 (19, 'Geordi', 60, 246),
 (20, 'Odo', 67, 220),
 (21, 'Miles', 19, 268),
 (23, 'Keiko', 51, 271),
 (25, 'Ben', 21, 445),
 (27, 'Leeta', 42, 363),
 (28, 'Martok', 49, 476),
 (29, 'Nog', 48, 364),
 (30, 'Keiko', 50, 175),
 (31, 'Miles', 39, 161),
 (32, 'Nog', 26, 281),
 (33, 'Dukat', 53, 197),
 (34, 'Jean-Luc', 43, 249),
 (35, 'Beverly', 27, 305),
 (39, 'Morn', 31, 192),
 (40, 'Odo', 52, 413),
 (41, 'Hugh', 67, 167),
 (43, 'Guinan', 58, 345),
 (44, 'Nerys', 35, 244),
 (49, 'Ezri', 40, 254),
 (50, 'Quark', 51, 283),
 (51, 'Lwaxana', 36, 212),
 (52, 'Beverly', 19, 269),
 (55, 'Keiko', 41, 278),
 (56, 'Gowron', 44, 194),
 (57, 'Odo', 57, 294),
 (58, '

In [ ]:
# 특정 나이에 속한 사람들의 친구 수 합계

In [22]:
#age,cnt
age_cnt_sum = struct_rdd.map(lambda row: (row[2], row[3])).reduceByKey(lambda a,b: a+b)
age_cnt_sum.take(3)

[(33, 3904), (26, 4115), (55, 3842)]

In [23]:
std_age = 30

age_cnt_sum.filter(lambda x:x[0]==std_age).collect()

[(30, 2594)]

In [ ]:
# 분석주제 추가